In [2]:
import sys
sys.path.append("..")

import datetime as dt
from ipywidgets import interact
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import statsmodels.api as sm
import tqdm
import vectorbtpro as vbt

import legitindicators as li
import pandas_ta as pta

from lib import utils, indicators, binance_loader

In [1121]:
df = binance_loader.load_ohlc_in_date_range('TRXUSDT', '2024-01-01', '2024-12-31')

In [1122]:
freq = '5min'
df = df.resample(freq).agg({'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum', 'quoteVolume':'sum', 'nTrades':'sum', 'upVolume':'sum', 'upQuoteVolume':'sum'})

In [1123]:
df['vold'] = 2*df['upVolume'] - df['volume']
df['volma'] = indicators.SuperSmoother(df['volume'], 20)
df['rvold'] = df['vold']/df['volma']

In [1124]:
df['return'] = df['close'].pct_change()
df['return_ma'] = indicators.SuperSmoother(df['return'].fillna(0).abs(), 20)
df['rreturn'] = df['return']/df['return_ma']

In [1125]:
df['aggr'] = (df['vold']*df['return'] > 0)

In [1126]:
df['ssmooth'] = indicators.SuperSmoother(df['close'], 50)
df['rms'] = np.sqrt((df['close'] - df['ssmooth']).pow(2).rolling(50).sum()/50)
df['band_u'] = df['ssmooth'] + 3*df['rms']
df['band_l'] = df['ssmooth'] - 3*df['rms']

In [ ]:
fig = go.FigureWidget(make_subplots(rows=3, cols=1, shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]))
fig.add_trace(go.Candlestick(), row=1, col=1)
fig.add_trace(go.Scatter(), row=2, col=1)
fig.add_trace(go.Scatter(), row=3, col=1)
fig.add_trace(go.Scatter(), row=1, col=1)
fig.add_trace(go.Scatter(), row=1, col=1)
fig.update_layout(height=600, margin=dict(l=20,r=20,b=20,t=20), xaxis=dict(rangeslider=dict(visible=False)))

@interact(date=np.unique(df.index.date), col=df.columns, col2=df.columns)
def update(date, col, col2):
   with fig.batch_update():
      _sdf = df.loc[str(date)]
      fig.data[0].x, fig.data[0].open, fig.data[0].high = _sdf.index, _sdf['open'], _sdf['high']
      fig.data[0].low, fig.data[0].close = _sdf['low'], _sdf['close']
      fig.data[1].x, fig.data[1].y = _sdf.index, _sdf[col]
      fig.data[2].x, fig.data[2].y = _sdf.index, _sdf[col2]
      fig.data[3].x, fig.data[3].y = _sdf.index, _sdf['band_u']
      fig.data[4].x, fig.data[4].y = _sdf.index, _sdf['band_l']
      fig.update_layout()
fig


In [1128]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
ee = EllipticEnvelope(support_fraction=1)
train_window = 5
dates = np.unique(df.index.date)

df['pred'] = np.nan
for i in tqdm.tqdm(range(train_window+1, len(dates))):
    train_df = df.loc[str(dates[i-train_window]):str(dates[i-1])]
    pred_df = df.loc[str(dates[i])]
    
    ee.fit(train_df[['return', 'vold']])
    pred = pd.Series(ee.decision_function(pred_df[['return', 'vold']]), index=pred_df.index)
    df.loc[pred_df.index, 'pred'] = pred

In [1130]:
# # Outlier Absorption
# le = df['pred'] < -25
# le &= ~df['aggr']
# le &= df['return'] < 0
# le &= df['low'] < df['band_l']

# se = df['pred'] < -25
# se &= ~df['aggr']
# se &= df['return'] > 0
# se &= df['high'] > df['band_u']

# le.sum(), se.sum()

In [ ]:
# Outlier Aggression
le = df['pred'] < -200
le &= df['aggr']
le &= df['return'] < 0
le &= df['close'] < df['band_l']

se = df['pred'] < -200
se &= df['aggr']
se &= df['return'] > 0
se &= df['close'] > df['band_u']

le.sum(), se.sum()

In [1153]:
df['le'] = le
df['se'] = se

In [1154]:
df['tr'] = pta.true_range(df['high'], df['low'], df['close'])
df['atr'] = indicators.SuperSmoother(df['tr'].fillna(0), 28)
df['natr'] = df['atr']/df['close']

In [1155]:
tpsl_mults = np.arange(0.5, 5, 0.5)
pf = vbt.Portfolio.from_signals(
    df['close'], open=df['open'], high=df['high'], low=df['low'],
    entries=le,
    # short_entries=se,
    freq=freq,
    # td_stop=13,
    # time_delta_format=0,
    sl_stop=vbt.Param([x*df['natr'] for x in tpsl_mults]),
    tp_stop=vbt.Param([x*df['natr'] for x in tpsl_mults]),
    upon_opposite_entry='Reverse',
)

In [1156]:
# stat_result = pf.trades.get_profit_factor().unstack()
stat_result = pf.sharpe_ratio.unstack()
stat_result.index = stat_result.index.str[7:].astype(int)
stat_result.columns = stat_result.columns.str[7:].astype(int)
stat_result = stat_result.sort_index().sort_index(axis=1)
stat_result.index = tpsl_mults
stat_result.columns = tpsl_mults
stat_result = stat_result.replace({np.inf:np.nan})
stat_result.index = stat_result.index.round(2)
stat_result.columns = stat_result.columns.round(2)

In [ ]:
ax = sns.heatmap(stat_result, annot=False)
ax.set_title('Sharpe Ratio by SL and TP (%)')

In [1167]:
tpsl_mults = np.arange(0.5, 5, 0.5)
pf = vbt.Portfolio.from_signals(
    df['close'], open=df['open'], high=df['high'], low=df['low'],
    entries=le,
    short_entries=se,
    freq=freq,
    # td_stop=13,
    # time_delta_format=0,
    sl_stop=4*df['natr'],
    tp_stop=3*df['natr'],
)

In [ ]:
pf.stats()

In [ ]:
pf.value.plot()

In [1173]:
records = pf.trades.records
records['dt'] = df.index[records['entry_idx']]
records['exit_dt'] = df.index[records['exit_idx']]
records['sl'] = 2*df['natr'].shift().iloc[records['entry_idx']].values
records['realized_r'] = records['return']/records['sl']
records = records.set_index('dt')

In [ ]:
records['realized_r'].cumsum().plot()